In [1]:
import os
import importlib
import numpy as np
import torch
from PIL import Image
from wordcloud import WordCloud

# os.chdir('/path/to/your/Clip')
!pwd
!echo $CUDA_VISIBLE_DEVICES
!nvidia-smi

/home/chenyun/work1/Clip
7
Mon Aug 23 02:11:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:1A:00.0 Off |                  Off |
| 41%   64C    P2   142W / 260W |  46729MiB / 48601MiB |     26%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     Off  | 00000000:1C:00.0 Off |     

In [2]:
from utils.clip_encoder import ClipEncoder
clip_encoder = ClipEncoder()

ClipEncoder ready.


In [4]:
from dtd2.data_api.dataset_api import TextureDescriptionData
from dtd2.models.layers.util import print_tensor_stats
from dtd2.data_api.eval_retrieve import retrieve_eval as dtd2_eval

split = 'test'
dataset = TextureDescriptionData()
print('dataset ready.')
img_num = len(dataset.img_splits[split])
phrase_num = len(dataset.phrases)

TextureDescriptionData ready. 
655 phrases with frequency above 10.
Image count: train 3222, val 805, test 1342
dataset ready.


In [5]:
eg = dataset.get_split_data('test')
print(eg)

{'image_name': 'honeycombed/honeycombed_0043.jpg', 'category': 'honeycombed', 'descriptions': ['old, hexagon, glass, honeycomb, delicate, pattern', 'cold, smooth, regular, shiny, sleek', 'honeycombed, interwoven, round, large, hexagonal', 'painted honeycombed pattern that is bronze along the ridges and gray within the honeycomb'], 'phrase_ids': {4, 101, 8, 41, 268, 269, 237, 77, 274, 275, 182, 56, 62, 126, -1, 191}}


In [6]:
cat_phrases = dict()
for img_name in dataset.img_splits['train']:
    img_data = dataset.img_data_dict[img_name]
    if img_data['category'] not in cat_phrases:
        cat_phrases[img_data['category']] = np.zeros(656)
    for p_id in img_data['phrase_ids']:
        if p_id >= 0:
            cat_phrases[img_data['category']][p_id] += 1


In [7]:
ave_phrases = np.zeros(656)
for p_count in cat_phrases.values():
    ave_phrases += p_count
ave_phrases /= len(cat_phrases)

# for p_num in [1, 5, 10, 15, 20, 25, 30, 35, 40]:
for p_num in [20]:
    cat_desc = dict()
    for cat, p_count in cat_phrases.items():
        p_scores = (p_count - ave_phrases) / len(dataset.img_splits['train'])
        pi_sorted = np.argsort(p_scores)
        phrases = list()
        for i in range(p_num):
            pi = pi_sorted[-i - 1]
            if p_scores[pi] < 0:
                print('WARN: score < 0: %s, %d' % (cat, pi))
                break
            phrases.append(dataset.phid_to_phrase(pi))
        cat_desc[cat] = 'An image of %s texture.' % ', '.join(phrases)
        print(cat, cat_desc[cat])
#     print(cat_desc)
    categories = sorted(list(cat_desc.keys()))
    cat_to_ids = {c: i for i, c in enumerate(categories)}
    text_inputs = [cat_desc[c] for c in categories]
    text_feats = clip_encoder.encode_text_list(text_inputs)
    correct = 0
    for img_name in dataset.img_splits['test']:
        img = dataset.load_img(img_name)
        img_feat = clip_encoder.encode_img(img)
        score = img_feat @ text_feats.T
        pred_ci = int(torch.argmax(score))
        if pred_ci == cat_to_ids[img_name.split('/')[0]]:
            correct += 1
    acc = correct / len(dataset.img_splits['test'])
    print(p_num, acc)

braided An image of braided, woven, twisted, knotted, interlaced, intertwined, bumpy, wicker, brown, interlocked, plaited, weaved, rope, basket, shiny, tough, tied, grooved, interwoven, brown color texture.
marbled An image of marbled, smooth, hard, cold, streaked, veined, marble, white, cracked, swirled, stone, mottled, solid, speckled, orange, gray, scratched, blotchy, striated, shiny texture.
woven An image of woven, interlaced, intertwined, wicker, wooden, brown, braided, interlocked, basket, rough, tan, handmade, knitted, knit, weaved, tight, fuzzy, twisted, squares, tightly wound texture.
swirly An image of swirly, whirly, twirly, circular, swirls, curly, red, black, green, white, abstract, spiralled, bright, pink, purple, curvy, colorful, swirled, yellow, painting texture.
meshed An image of meshed, metallic, metal, silver, netted, linked, chain, woven, cold, shiny, diamond, wire, fenced, knotted, iron, diamond shaped, intertwined, mesh, diamond shape, holey texture.
wrinkled An

In [54]:
texts = ['An image of %s texture.' % c for c in categories]
text_feats = clip_encoder.encode_text_list(texts)
correct = 0
for img_name in dataset.img_splits['test']:
    img = dataset.load_img(img_name)
    img_feat = clip_encoder.encode_img(img)
    score = img_feat @ text_feats.T
    pred_ci = int(torch.argmax(score))
    if pred_ci == cat_to_ids[img_name.split('/')[0]]:
        correct += 1
acc = correct / len(dataset.img_splits['test'])

In [55]:
print(acc)

0.4105812220566319
